In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pathlib 
import os
from scipy import signal


import torch
# from torch import nn
# from torch.utils.data import DataLoader

import PT_files.save_load as sl
from DnCNN_NP.layers  import relu#, np_BatchNorm2d, np_Conv2d

In [2]:
def pytorch_numpy_comparison(input_data,
                             pytorch_output,
                             numpy_output,
                             sample_idx):
    
    
    
    fig, ax = plt.subplots(1, 3, figsize=(24,20))
    vmin, vmax = np.percentile(input_data[sample_idx], (1,99))
    # vmin, vmax = np.percentile(pytorch_output[sample][feature_map], (1,99))


    ax[0].imshow(pytorch_output[sample_idx][0], vmin=vmin, vmax=vmax, origin='lower', interpolation='none')
    ax[0].axis('off')
    ax[0].set_title('Pytorch BatchNorm', fontsize=30)
    ax[1].imshow(input_data[sample_idx][0],vmin=vmin, vmax=vmax, origin='lower', interpolation='none')
    ax[1].axis('off')
    ax[1].set_title('Input Sample', fontsize=30)
    ax[2].imshow(numpy_output[sample_idx][0], vmin=vmin, vmax=vmax, origin='lower', interpolation='none')
    ax[2].axis('off')
    ax[2].set_title('Numpy BatchNorm', fontsize=30)
    
    
def full_grid_pass_window(dataset, model, model_params, samp_idx):
    
    full = np.empty((1, 1, 6000,6000))
    count = np.empty((1, 1, 6000,6000))
    
    noise_data = dataset[0]
    param_name = model_params
    print('pass 1')
    
    current_dir = pathlib.Path().resolve()
    model_params_path = current_dir / 'Model_params'
    assert model_params_path.exists()
    model_path = model_params_path / param_name
    print('pass 2')
    
    model = model()
    model.to(device)
    model.load_state_dict(torch.load(str(model_path)))
    model.eval();
    print('pass 3')
    # telling pytorch this is for inference and not learning, so keeps the weights unchanged
    with torch.no_grad():
        
        print('pass 4')
        torch.cuda.empty_cache()
        test_noise = torch.as_tensor(noise_data[samp_idx:samp_idx+1,:,:, :])
        test_noise = test_noise.to(device)

        print('pass 5')
        output = model(test_noise)
        print('pass 6')
        resid_img = output.detach().cpu().numpy()
        print('pass 7')

        full[:, :, :, :] += resid_img

    return full

In [3]:
PATH = pathlib.Path(os.getenv('PSCRATCH'))
DATA = PATH / 'DESI_dn' /'Model_params'
assert DATA.exists()
name = '6k_model_wb_e800_lys20_58feat.pth'

# weights = np.load(DATA / name)
weights = torch.load(str(DATA / name))


#Load the actual data that we're working on & print the shape of this data
test_data = sl.NERSC_load('test_data_40%_6000.npy')
sample = test_data[0]
print('Shape of test set=', sample.shape)

Shape of test set= (108, 1, 6000, 6000)


In [4]:
# def np_Conv2d(input_data, weights_dict, prefix, stride=1, padding="same", dilation=1):
    
#     batch_size, input_channels, height, width = input_data.shape # (N, Cin, Hin, Win)
#     kernel_size = weights_dict[str(prefix) + 'weight'][0][0].shape
#     output_channels = len(weights_dict[str(prefix) + 'weight'])
    
#     # Convert string padding into numerical padding
#     # Using strings allow for one variable to account for padding & mode (see signal.correlated2d)
#     mode = padding
#     if mode == "same":
#         padding = 1
#     elif mode == "valid":
#         padding = 0
    
#     height_out = ((height + (2*padding) - dilation * (kernel_size[0] - 1) - 1) / stride) + 1
#     height_out = int(height_out)
#     width_out = ((width + (2*padding) - dilation * (kernel_size[1] - 1) - 1) / stride) + 1
#     width_out = int(width_out)

#     output = np.empty((batch_size, output_channels, height_out, width_out))
    
#     for i in range(batch_size):
#         for j in range(output_channels):
#             output[i, j, :, :] = weights_dict[str(prefix) + 'bias'][j].detach().cpu().numpy() + signal.correlate2d(input_data[i][0], weights_dict[str(prefix) + 'weight'][j][0].detach().cpu().numpy(), mode=mode)
               
#     return output

def np_Conv2d(input_data, weights_dict, prefix, stride=1, padding="same", dilation=1):
    
    batch_size, input_channels, height, width = input_data.shape # (N, Cin, Hin, Win)
    
    weight = weights_dict[str(prefix) + 'weight']
    bias = weights_dict[str(prefix) + 'bias']
    
    kernel_size = weight[0][0].shape
    output_channels = len(weight)
    
    # Convert string padding into numerical padding
    # Using strings allow for one variable to account for padding & mode (see signal.correlated2d)
    mode = padding
    if mode == "same":
        padding = 1
    elif mode == "valid":
        padding = 0
    
    height_out = ((height + (2*padding) - dilation * (kernel_size[0] - 1) - 1) / stride) + 1
    height_out = int(height_out)
    width_out = ((width + (2*padding) - dilation * (kernel_size[1] - 1) - 1) / stride) + 1
    width_out = int(width_out)

    output = np.empty((batch_size, output_channels, height_out, width_out))
    
    for i in range(batch_size):
        for j in range(output_channels):
            output[i, j, :, :] = weight[j].detach().cpu().numpy() + signal.correlate2d(input_data[i][0], bias[j][0].detach().cpu().numpy(), mode=mode)
    
    return output


def np_BatchNorm2d(input_data, prefix, weights_dict, epsilon=1e-5):
    
    x = input_data
    
    gamma = weights_dict[str(prefix) + 'weight'].detach().cpu().numpy().reshape(-1, 1, 1)
    beta = weights_dict[str(prefix) + 'bias'].detach().cpu().numpy().reshape(-1, 1, 1)
    mean = weights_dict[str(prefix) + 'running_mean'].detach().cpu().numpy().reshape(-1, 1, 1)
    var = weights_dict[str(prefix) + 'running_var'].detach().cpu().numpy().reshape(-1, 1, 1)
        
        
    output = ((x - mean) / np.sqrt(var + epsilon)) * gamma + beta
    return output

In [5]:
def load_weights(data, prefix, weights_dict):
    
    if type(prefix) != str:
        raise Exception("The prefix must be a string!")
                        
    if prefix.endswith('0.'):
        conv_layer_output = np_Conv2d(input_data=data,
                                      weights_dict=weights_dict,
                                      prefix=prefix)
        return conv_layer_output
    
    elif prefix.endswith('1.'):
        batch_layer_output = np_BatchNorm2d(x=data,
                                            prefix=prefix,
                                            weights_dict=weights_dict)
        return batch_layer_output
    
    elif prefix.endswith('19.'):
        conv_layer_output = np_Conv2d(input_data=data,
                                      weights_dict=weights_dict,
                                      prefix=prefix)
        return conv_layer_output

In [6]:
# def np_Conv2d(input_data, weights_dict, prefix, stride=1, padding="same", dilation=1):
    
#     batch_size, input_channels, height, width = input_data.shape # (N, Cin, Hin, Win)
#     kernel_size = weights_dict[str(prefix) + 'weight'][0][0].shape
#     output_channels = len(weights_dict[str(prefix) + 'weight'])
    
#     # Convert string padding into numerical padding
#     # Using strings allow for one variable to account for padding & mode (see signal.correlated2d)
#     mode = padding
#     if mode == "same":
#         padding = 1
#     elif mode == "valid":
#         padding = 0
    
#     height_out = ((height + (2*padding) - dilation * (kernel_size[0] - 1) - 1) / stride) + 1
#     height_out = int(height_out)
#     width_out = ((width + (2*padding) - dilation * (kernel_size[1] - 1) - 1) / stride) + 1
#     width_out = int(width_out)

#     output = np.empty((batch_size, output_channels, height_out, width_out))
    
#     for i in range(batch_size):
#         for j in range(output_channels):
#             output[i, j, :, :] = weights_dict[str(prefix) + 'bias'][j].detach().cpu().numpy() + signal.correlate2d(input_data[i][0], weights_dict[str(prefix) + 'weight'][j][0].detach().cpu().numpy(), mode=mode)
    
#     return output


# prefix_str = 'layers.0.0.'

# %timeit conv_layer_output = np_Conv2d(input_data=sample[:0], weights_dict=weights, prefix=prefix_str)

In [7]:
def np_BatchNorm2d(input_data, prefix, weights_dict, epsilon=1e-5):
    
    x = input_data
    
    gamma = weights_dict[str(prefix) + 'weight'].detach().cpu().numpy().reshape(-1, 1, 1)
    beta = weights_dict[str(prefix) + 'bias'].detach().cpu().numpy().reshape(-1, 1, 1)
    mean = weights_dict[str(prefix) + 'running_mean'].detach().cpu().numpy().reshape(-1, 1, 1)
    var = weights_dict[str(prefix) + 'running_var'].detach().cpu().numpy().reshape(-1, 1, 1)
        
        
    output = ((x - mean) / np.sqrt(var + epsilon)) * gamma + beta
    return output


prefix_str = 'layers.1.1.'

%timeit batch_layer_output = np_BatchNorm2d(input_data=sample[:0], weights_dict=weights, prefix=prefix_str)

112 µs ± 120 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [8]:
def np_BatchNorm2d(input_data, prefix, weights_dict, epsilon=1e-5):
    
    x = input_data
    
    gamma = weights_dict[str(prefix) + 'weight'].detach().cpu().numpy().reshape(-1, 1, 1)
    beta = weights_dict[str(prefix) + 'bias'].detach().cpu().numpy().reshape(-1, 1, 1)
    mean = weights_dict[str(prefix) + 'running_mean'].detach().cpu().numpy().reshape(-1, 1, 1)
    var = weights_dict[str(prefix) + 'running_var'].detach().cpu().numpy().reshape(-1, 1, 1)
        
        
    output = ((x - mean) / np.sqrt(var + epsilon)) * gamma + beta
    return output


prefix_str = 'layers.1.1.'

# %timeit conv_layer_output = np_Conv2d(input_data=sample[:0], weights_dict=weights, prefix=prefix_str)

In [9]:
# class np_DnCNN():
#     """
#     A np_DnCNN model object. The model holds all of the relevant layer
#     information, including layer names, weights, and definitions.
    
#     Parameters:
#     -----------
#     weights: OrderedDict
#         Ordered dictionary that maps layers names to layer weights.
#     nlayers: int, optional
#         Number of layers this model was trained for. Defaults to 20.
#     """
#     def __init__(self, weights, nlayers=20):
#         # Store the weights to access later.
#         self.weights = weights
#         self.nlayers = nlayers
        
        

In [10]:
from collections import OrderedDict

# Replace the last part of the key that describes what layer it is
# part of and replaces it with empty space
layers_list = [x.replace('weight', '').replace('bias', '').replace('running_mean', '').replace('running_var', '').replace('num_batches_tracked', '') for x in weights.keys()]
# Convert this list which has duplicated elements due to removing
# identifying elements ie. for the first conv layer we had
# layers.0.0.weight & layers.0.0.bias, but now after removing them we
# have layers.0.0 & layers.0.0
# The code below deletes the duplicated elements
layers_list = list(OrderedDict.fromkeys(layers_list))
print(layers_list)

['layers.0.0.', 'layers.1.0.', 'layers.1.1.', 'layers.2.0.', 'layers.2.1.', 'layers.3.0.', 'layers.3.1.', 'layers.4.0.', 'layers.4.1.', 'layers.5.0.', 'layers.5.1.', 'layers.6.0.', 'layers.6.1.', 'layers.7.0.', 'layers.7.1.', 'layers.8.0.', 'layers.8.1.', 'layers.9.0.', 'layers.9.1.', 'layers.10.0.', 'layers.10.1.', 'layers.11.0.', 'layers.11.1.', 'layers.12.0.', 'layers.12.1.', 'layers.13.0.', 'layers.13.1.', 'layers.14.0.', 'layers.14.1.', 'layers.15.0.', 'layers.15.1.', 'layers.16.0.', 'layers.16.1.', 'layers.17.0.', 'layers.17.1.', 'layers.18.0.', 'layers.18.1.', 'layers.19.']


In [11]:
def model_flow(input_data, weights_dict, prefix_list):
    
    output = np_Conv2d(input_data=input_data,
                   weights_dict=weights_dict,
                   prefix=prefix_list[0])


    for i in range(len(prefix_list)-2):
        print(prefix_list[i+1])

        if prefix_list[i+1].endswith('0.'):

            output = np_Conv2d(input_data=output,
                               weights_dict=weights_dict,
                               prefix=prefix_list[i+1])

        elif layers_list[i+1].endswith('1.'):
            output = np_BatchNorm2d(input_data=output,
                                    weights_dict=weights_dict,
                                    prefix=prefix_list[i+1])

    output = np_Conv2d(input_data=output,
                       weights_dict=weights_dict,
                       prefix=prefix_list[-1])
    
    return output

%timeit output = model_flow(input_data=sample[:2], weights_dict=weights, prefix_list=layers_list)
# pdb.pm()

IndexError: invalid index of a 0-dim tensor. Use `tensor.item()` in Python or `tensor.item<T>()` in C++ to convert a 0-dim tensor to a number

In [19]:
# 1st layer
output = np_Conv2d(input_data=sample[:1],
                   weights_dict=weights,
                   prefix=layers_list[0])


for i in range(len(layers_list)-2):
    print(layers_list[i+1])
    
    if layers_list[i+1].endswith('0.'):
        
        output = np_Conv2d(input_data=output,
                           weights_dict=weights,
                           prefix=layers_list[i+1])
        
    elif layers_list[i+1].endswith('1.'):
        output = np_BatchNorm2d(input_data=output,
                                weights_dict=weights,
                                prefix=layers_list[i+1])
        
output = np_Conv2d(input_data=output,
                   weights_dict=weights,
                   prefix=layers_list[-1])

layers.1.0.
layers.1.1.
layers.2.0.


KeyboardInterrupt: 

In [ ]:
plt.imshow(output[0][0])

In [98]:
layers_list[-1]

'layers.19.'

In [97]:
layers_list[0].endswith('0.')

True

In [96]:
for i in range(len(layers_list)-2): # -2 to account for not looping through 1st & last layer
    print(layers_list[i+1]) # +1 to start the loop after the first layer

layers.1.0.
layers.1.1.
layers.2.0.
layers.2.1.
layers.3.0.
layers.3.1.
layers.4.0.
layers.4.1.
layers.5.0.
layers.5.1.
layers.6.0.
layers.6.1.
layers.7.0.
layers.7.1.
layers.8.0.
layers.8.1.
layers.9.0.
layers.9.1.
layers.10.0.
layers.10.1.
layers.11.0.
layers.11.1.
layers.12.0.
layers.12.1.
layers.13.0.
layers.13.1.
layers.14.0.
layers.14.1.
layers.15.0.
layers.15.1.
layers.16.0.
layers.16.1.
layers.17.0.
layers.17.1.
layers.18.0.
layers.18.1.


In [51]:
prefix_str = 'layers.0.0.'

%timeit conv_layer_output = np_Conv2d(input_data=sample[:2], weights_dict=weights, prefix=prefix_str)

1min 49s ± 94.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [52]:
conv_layer_output = np_Conv2d(input_data=sample[0],
                              weights_dict=weights,
                              prefix=prefix_str

odict_keys(['layers.0.0.weight', 'layers.0.0.bias', 'layers.1.0.weight', 'layers.1.0.bias', 'layers.1.1.weight', 'layers.1.1.bias', 'layers.1.1.running_mean', 'layers.1.1.running_var', 'layers.1.1.num_batches_tracked', 'layers.2.0.weight', 'layers.2.0.bias', 'layers.2.1.weight', 'layers.2.1.bias', 'layers.2.1.running_mean', 'layers.2.1.running_var', 'layers.2.1.num_batches_tracked', 'layers.3.0.weight', 'layers.3.0.bias', 'layers.3.1.weight', 'layers.3.1.bias', 'layers.3.1.running_mean', 'layers.3.1.running_var', 'layers.3.1.num_batches_tracked', 'layers.4.0.weight', 'layers.4.0.bias', 'layers.4.1.weight', 'layers.4.1.bias', 'layers.4.1.running_mean', 'layers.4.1.running_var', 'layers.4.1.num_batches_tracked', 'layers.5.0.weight', 'layers.5.0.bias', 'layers.5.1.weight', 'layers.5.1.bias', 'layers.5.1.running_mean', 'layers.5.1.running_var', 'layers.5.1.num_batches_tracked', 'layers.6.0.weight', 'layers.6.0.bias', 'layers.6.1.weight', 'layers.6.1.bias', 'layers.6.1.running_mean', 'layer

In [21]:
prefix = 'layers.0.0.'
end = 'weight'

# print(prefix + end)

weights[prefix + end].shape

torch.Size([58, 1, 3, 3])

In [58]:
weights.keys()

odict_keys(['layers.0.0.weight', 'layers.0.0.bias', 'layers.1.0.weight', 'layers.1.0.bias', 'layers.1.1.weight', 'layers.1.1.bias', 'layers.1.1.running_mean', 'layers.1.1.running_var', 'layers.1.1.num_batches_tracked', 'layers.2.0.weight', 'layers.2.0.bias', 'layers.2.1.weight', 'layers.2.1.bias', 'layers.2.1.running_mean', 'layers.2.1.running_var', 'layers.2.1.num_batches_tracked', 'layers.3.0.weight', 'layers.3.0.bias', 'layers.3.1.weight', 'layers.3.1.bias', 'layers.3.1.running_mean', 'layers.3.1.running_var', 'layers.3.1.num_batches_tracked', 'layers.4.0.weight', 'layers.4.0.bias', 'layers.4.1.weight', 'layers.4.1.bias', 'layers.4.1.running_mean', 'layers.4.1.running_var', 'layers.4.1.num_batches_tracked', 'layers.5.0.weight', 'layers.5.0.bias', 'layers.5.1.weight', 'layers.5.1.bias', 'layers.5.1.running_mean', 'layers.5.1.running_var', 'layers.5.1.num_batches_tracked', 'layers.6.0.weight', 'layers.6.0.bias', 'layers.6.1.weight', 'layers.6.1.bias', 'layers.6.1.running_mean', 'layer

In [83]:
layers_list = [x.replace('weight', '').replace('bias', '').replace('running_mean', '').replace('running_var', '').replace('num_batches_tracked', '') for x in weights.keys()]
layers_list[5]

'layers.1.1.'

['layers.0.0.',
 'layers.1.0.',
 'layers.1.1.',
 'layers.2.0.',
 'layers.2.1.',
 'layers.3.0.',
 'layers.3.1.',
 'layers.4.0.',
 'layers.4.1.',
 'layers.5.0.',
 'layers.5.1.',
 'layers.6.0.',
 'layers.6.1.',
 'layers.7.0.',
 'layers.7.1.',
 'layers.8.0.',
 'layers.8.1.',
 'layers.9.0.',
 'layers.9.1.',
 'layers.10.0.',
 'layers.10.1.',
 'layers.11.0.',
 'layers.11.1.',
 'layers.12.0.',
 'layers.12.1.',
 'layers.13.0.',
 'layers.13.1.',
 'layers.14.0.',
 'layers.14.1.',
 'layers.15.0.',
 'layers.15.1.',
 'layers.16.0.',
 'layers.16.1.',
 'layers.17.0.',
 'layers.17.1.',
 'layers.18.0.',
 'layers.18.1.',
 'layers.19.']

In [76]:
res = [i for n, i in enumerate(layers_list) if i not in layers_list[:n]]
res[:5]

['layers.6.0.', 'layers.6.1.', 'layers.0.0.', 'layers.2.0.', 'layers.7.0.']

In [71]:
layers_list = list(set(layers_list))
layers_list

['layers.6.0.',
 'layers.6.1.',
 'layers.0.0.',
 'layers.2.0.',
 'layers.7.0.',
 'layers.11.0.',
 'layers.12.0.',
 'layers.3.1.',
 'layers.8.0.',
 'layers.15.0.',
 'layers.5.0.',
 'layers.1.1.',
 'layers.13.1.',
 'layers.17.0.',
 'layers.19.',
 'layers.8.1.',
 'layers.4.1.',
 'layers.9.0.',
 'layers.10.0.',
 'layers.10.1.',
 'layers.17.1.',
 'layers.18.1.',
 'layers.16.1.',
 'layers.13.0.',
 'layers.15.1.',
 'layers.5.1.',
 'layers.18.0.',
 'layers.4.0.',
 'layers.9.1.',
 'layers.11.1.',
 'layers.14.0.',
 'layers.12.1.',
 'layers.2.1.',
 'layers.3.0.',
 'layers.16.0.',
 'layers.7.1.',
 'layers.14.1.',
 'layers.1.0.']

In [57]:
'queen'.find(

<function str.find>

In [6]:
'queen'.find

<function str.find>

Some good [string methods](https://docs.python.org/2.5/lib/string-methods.html)

or these [Python 3 versions](https://docs.python.org/3/library/stdtypes.html#string-methods)

In [7]:
weights.keys()

odict_keys(['layers.0.0.weight', 'layers.0.0.bias', 'layers.1.0.weight', 'layers.1.0.bias', 'layers.1.1.weight', 'layers.1.1.bias', 'layers.1.1.running_mean', 'layers.1.1.running_var', 'layers.1.1.num_batches_tracked', 'layers.2.0.weight', 'layers.2.0.bias', 'layers.2.1.weight', 'layers.2.1.bias', 'layers.2.1.running_mean', 'layers.2.1.running_var', 'layers.2.1.num_batches_tracked', 'layers.3.0.weight', 'layers.3.0.bias', 'layers.3.1.weight', 'layers.3.1.bias', 'layers.3.1.running_mean', 'layers.3.1.running_var', 'layers.3.1.num_batches_tracked', 'layers.4.0.weight', 'layers.4.0.bias', 'layers.4.1.weight', 'layers.4.1.bias', 'layers.4.1.running_mean', 'layers.4.1.running_var', 'layers.4.1.num_batches_tracked', 'layers.5.0.weight', 'layers.5.0.bias', 'layers.5.1.weight', 'layers.5.1.bias', 'layers.5.1.running_mean', 'layers.5.1.running_var', 'layers.5.1.num_batches_tracked', 'layers.6.0.weight', 'layers.6.0.bias', 'layers.6.1.weight', 'layers.6.1.bias', 'layers.6.1.running_mean', 'layer

In [8]:
for i in range(20):
    layers = [k for k in weights.keys() if k.startswith("layers."+str(i))]
layers

['layers.19.weight', 'layers.19.bias']

In [9]:
for i in range(20):
    for k in weights.keys():
        layers = k.strip("layers."+str(i))
layers

'bi'

In [10]:
y = []
for i in weights.keys():
    if 'layers.1.' in i:
        y.append(i)
        
y

['layers.1.0.weight',
 'layers.1.0.bias',
 'layers.1.1.weight',
 'layers.1.1.bias',
 'layers.1.1.running_mean',
 'layers.1.1.running_var',
 'layers.1.1.num_batches_tracked']

In [11]:
y

['layers.1.0.weight',
 'layers.1.0.bias',
 'layers.1.1.weight',
 'layers.1.1.bias',
 'layers.1.1.running_mean',
 'layers.1.1.running_var',
 'layers.1.1.num_batches_tracked']

In [12]:
lys = []
for i in weights.keys():
    # str.lstrip: Return a copy of the string with trailing characters removed.
    #The chars argument is a string specifying the set of characters to be removed.
    #If omitted or None, the chars argument defaults to removing whitespace.
    #The chars argument is not a suffix; rather, all combinations of its values are stripped:
    lys.append(i.strip('layers.0123456789'))
    
print(lys)

['weight', 'bi', 'weight', 'bi', 'weight', 'bi', 'unning_mean', 'unning_v', 'num_batches_tracked', 'weight', 'bi', 'weight', 'bi', 'unning_mean', 'unning_v', 'num_batches_tracked', 'weight', 'bi', 'weight', 'bi', 'unning_mean', 'unning_v', 'num_batches_tracked', 'weight', 'bi', 'weight', 'bi', 'unning_mean', 'unning_v', 'num_batches_tracked', 'weight', 'bi', 'weight', 'bi', 'unning_mean', 'unning_v', 'num_batches_tracked', 'weight', 'bi', 'weight', 'bi', 'unning_mean', 'unning_v', 'num_batches_tracked', 'weight', 'bi', 'weight', 'bi', 'unning_mean', 'unning_v', 'num_batches_tracked', 'weight', 'bi', 'weight', 'bi', 'unning_mean', 'unning_v', 'num_batches_tracked', 'weight', 'bi', 'weight', 'bi', 'unning_mean', 'unning_v', 'num_batches_tracked', 'weight', 'bi', 'weight', 'bi', 'unning_mean', 'unning_v', 'num_batches_tracked', 'weight', 'bi', 'weight', 'bi', 'unning_mean', 'unning_v', 'num_batches_tracked', 'weight', 'bi', 'weight', 'bi', 'unning_mean', 'unning_v', 'num_batches_tracked',

In [13]:
lys = []
for i in weights.keys():
    # str.lstrip: Return a copy of the string with trailing characters removed.
    #The chars argument is a string specifying the set of characters to be removed.
    #If omitted or None, the chars argument defaults to removing whitespace.
    #The chars argument is not a suffix; rather, all combinations of its values are stripped:
    lys.append(i.lstrip('layers.'))
    
print(lys)

['0.0.weight', '0.0.bias', '1.0.weight', '1.0.bias', '1.1.weight', '1.1.bias', '1.1.running_mean', '1.1.running_var', '1.1.num_batches_tracked', '2.0.weight', '2.0.bias', '2.1.weight', '2.1.bias', '2.1.running_mean', '2.1.running_var', '2.1.num_batches_tracked', '3.0.weight', '3.0.bias', '3.1.weight', '3.1.bias', '3.1.running_mean', '3.1.running_var', '3.1.num_batches_tracked', '4.0.weight', '4.0.bias', '4.1.weight', '4.1.bias', '4.1.running_mean', '4.1.running_var', '4.1.num_batches_tracked', '5.0.weight', '5.0.bias', '5.1.weight', '5.1.bias', '5.1.running_mean', '5.1.running_var', '5.1.num_batches_tracked', '6.0.weight', '6.0.bias', '6.1.weight', '6.1.bias', '6.1.running_mean', '6.1.running_var', '6.1.num_batches_tracked', '7.0.weight', '7.0.bias', '7.1.weight', '7.1.bias', '7.1.running_mean', '7.1.running_var', '7.1.num_batches_tracked', '8.0.weight', '8.0.bias', '8.1.weight', '8.1.bias', '8.1.running_mean', '8.1.running_var', '8.1.num_batches_tracked', '9.0.weight', '9.0.bias', '9.

In [14]:
for i in range(len(lys)):
    if 'bias' in lys[i]:
        print(3)

3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


In [15]:

ly1 = []

if lys[0].startswith('0') == True:
    ly1.append(

SyntaxError: unexpected EOF while parsing (1302659768.py, line 4)

In [ ]:
for i in range(20):
    lys.startswith(

In [ ]:
for i in range(20):
    layers = {'layer'+str(i): lys}

In [ ]:
lys = []
for i in weights.keys():
    # str.lstrip: Return a copy of the string with trailing characters removed.
    #The chars argument is a string specifying the set of characters to be removed.
    #If omitted or None, the chars argument defaults to removing whitespace.
    #The chars argument is not a suffix; rather, all combinations of its values are stripped:
    lys.append(i.removeprefix('layers.0123456789'))
    
print(lys)

In [ ]:
conv0_weights = np.asarray(weights['layers.0.0.weight'].detach().cpu())

In [ ]:
conv0_weights.shape

In [ ]:
# training_data = sl.NERSC_load('training_data_60%_6000.npy')        
test_data = sl.NERSC_load('test_data_40%_6000.npy')

sample = test_data[0][0]

```python
# Indices of the data that will be given to the numpy DnCNN model
print(test_data.shape) 
>>>(2, 108, 1, 6000, 6000)
-> (noisy/clean pair, num of samps, num of channels, height, width)

print(test_data[0].shape) # noisy sample chosen. test_data[1]==clean
>>>(108, 1, 6000, 6000)
-> (num of samps, num of channels, height, width)

print(test_data[0][0].shape) # first noisy image of noisy sample chosen
>>>(1, 6000, 6000)
-> (num of channels, height, width)
```

In [ ]:
seed = 123
rng = np.random.RandomState(seed)

In [ ]:
start_idx=2000
end_idx=2200

fig, ax = plt.subplots(1, 1, figsize=(14,10))
vmin, vmax = np.percentile(sample[0][start_idx:end_idx,start_idx:end_idx], (1,99))


ax.imshow(sample[0][start_idx:end_idx,start_idx:end_idx],
             vmin=vmin, vmax=vmax, origin='lower', interpolation='none')
ax.axis('off')
ax.set_title('Noisy Sample', fontsize=30)


fig, ax = plt.subplots(1, 1, figsize=(14,10))
vmin, vmax = np.percentile(sample[0][start_idx:end_idx,start_idx:end_idx], (1,99))


ax.imshow(sample[0][0:6000,0:2000],
             vmin=vmin, vmax=vmax, origin='lower', interpolation='none')
ax.axis('off')
ax.set_title('Noisy Sample', fontsize=30)

In [ ]:
sample.shape # one channel, 6000 pixel height, by 6000 pixel width

# for conv2d in pytorch padding=1 is ~ padding="same"

Good discussion found [here](https://stackoverflow.com/questions/62166719/padding-same-conversion-to-pytorch-padding#:~:text=In%20PyTorch%20you%20can%20directly,~%20%22same%22%20in%20keras.)

Difference of keras/tf `batch_normalization` and pytorch's `batch_norm2d`

DnCNN uses `Conv2D`, `BatchNorm2d`, and `ReLU` layers

In [ ]:
# from the nunmpy conv layer vid
# input-shape = (depth, height, width)
# kernel_size = size of each matrix inside each kernel
# depth = num of kernels we want and hence the depth of the output

In [ ]:

# batchnorm weights
'layers.1.1.weight', 
'layers.1.1.bias', 
'layers.1.1.running_mean', 
'layers.1.1.running_var', 
'layers.1.1.num_batches_tracked'

In [ ]:
print('Weights =', weights['layers.1.1.weight'])
print('Biases =', weights['layers.1.1.bias'])
print('Mean =', weights['layers.1.1.running_mean'])
print('var =', weights['layers.1.1.running_var'])
print('num_batches_tracked(?) =', weights['layers.1.1.num_batches_tracked'])

In [ ]:
nn.Conv2d

In [ ]:
# Activations
relu = lambda x: np.maximum(0, x)

# Required Layers

def BatchNorm2d(x, mean, var, beta, gamma, epsilon=1e-5):
    """
    Computes the batch normalized version of the input.
    
    This function implements a batch normalization layer. Batch normalization
    renormalizes the input to the layer to a more parsable data range.
    
    Parameters:
    -----------
    x: numpy.ndarray
        Input image data.
    mean: numpy.ndarray
        Running mean of the dataset, computed during training.
    var: numpy.ndarray
        Running variance of the dataset, computed during training.
    beta: numpy.ndarray
        Offset value added to the normalized output.
        (These are the biases from the model parameter dictionary).
    gamma: numpy.ndarray
        Scale value to rescale the normalzied output.
        (These are the weights from the model parameter dictionary).
    epsilon: float
        Small constant for numerical stability. 
        Default = 1e-5.
        
    Returns:
    --------
    numpy.ndarray
        Output of the batch normalization.
        
    Notes:
    ------
    The operation implemented in this function is:
    
    .. math:: \\frac{\gamma (x - \mu)}{\sigma + \epsilon} + \\beta
    
    where :math:`\mu` is the running mean of the dataset and :math:`\sigma` is
    the running variance of the dataset, both of which are computed during
    training.
    
    For more details and documentation on the PyTorch BatchNorm2d function
    that this function mimics can be found at 
    https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html
    """
    return ((x - mean) / np.sqrt(var + epsilon)) * gamma + beta


def Conv2d():
    pass

In [ ]:
print(test_data[0].shape)
print(sample.shape)

In [ ]:
weights['layers.1.1.weight'])
print('Biases =', weights['layers.1.1.bias'])
print('Mean =', weights['layers.1.1.running_mean'])
print('var =', weights['layers.1.1.running_var'])

In [ ]:
BatchNorm2d(x=sample,
            mean=weights['layers.1.1.running_mean'].detach().cpu().numpy(),
            var=weights['layers.1.1.running_var'].detach().cpu().numpy(),
            gamma=weights['layers.1.1.weight'].detach().cpu().numpy(),
            beta=weights['layers.1.1.bias'].detach().cpu().numpy())